In [1]:

# coding: utf-8

# In[1]:



import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
_SAVEDIR_ = 'Tex/figures/'


def prog_bar(cur_val,max_val,msg = ""):
    import sys
    sys.stdout.write('\r')
    eq = int(np.ceil(np.true_divide(cur_val*100,max_val*5)))
    sys.stdout.write("[{:20s}] {}/{} {:}  ".format('='*eq, cur_val+1,max_val,msg))
    sys.stdout.flush()


# In[99]:


# Define parameters
G = -4
visc = 1/6
dt = 1
dx = 1
rho_psi = 1
rho_in = 1
omega = 1/(3.*visc*(dt*dt)/(dx*dx) + 0.5*dt) # this is just one....
lx = 100
ly = lx
Nsteps = 300
cs=1./np.sqrt(3)


# In[133]:


def init(rho_in):
    f = np.zeros([Nsteps, 9,lx,ly])
    
#     rho = np.ones([lx,ly])*rho_in
#     rho *= 1+.01*(np.random.rand(lx,ly)-.5)*2
    rho = np.zeros([lx,ly])
    disp_x = lx/2
    disp_y = ly/2
    y,x = np.ogrid[-disp_x:lx-disp_x,-disp_y:ly-disp_y]
    radius = 3
    mask = x*x +y*y < radius**2
    rho[mask] = 2.4
    rho *= 1+.05*(np.random.rand(lx,ly)-.5)*2
    rho[mask == 0] = .125
    
    u = np.zeros([lx,ly])
    v = np.zeros([lx,ly])
    for kk in range(0,9):  
        f[0,kk,:,:] = w[kk]*rho
    return f, rho, u ,v
        
    # Init rho with perturbation
def calc_macro(f,rho,u,v):
    # f should be at a given time step
    u[:,:] = 0
    v[:,:] = 0
    
    rho = np.sum(f,axis=0)
    for k in range(9):
        u += f[k]*ex[k]
#         print(ey[k])
        v += f[k]*ey[k]
    u=u/rho
    v=v/rho
    return rho, u ,v
def force(rho, u, v):
    # omega, and rho_psi are global variables
    psi = rho_psi*(1-np.exp(-rho/rho_psi))
    # a negative roll velocity implied i+1

    for i in range(lx):
        for j in range(ly):
            f_x=psi[i,j]*(psi[(i+1)%lx,j]-psi[(i-1)%lx,j])
            f_y=psi[i,j]*(psi[i,(j+1)%ly]-psi[i,(j-1)%ly])
            f2_x=psi[i,j]*(psi[(i+1)%lx,(j-1)%ly]+psi[(i+1)%lx,(j+1)%ly]-psi[(i-1)%lx,(j-1)%ly]-psi[(i-1)%lx,(j+1)%ly])
            f2_y=psi[i,j]*(-psi[(i+1)%lx,(j-1)%ly]+psi[(i+1)%lx,(j+1)%ly]-psi[(i-1)%lx,(j-1)%ly]+psi[(i-1)%lx,(j+1)%ly])
    force_x = -G*(f_x*1/9+f2_x*1/36)
    force_y = -G*(f_y*1/9+f2_y*1/36)


    # shift equlibrium
    u += force_x/(omega*rho)
    v += force_y/(omega*rho)
    return u, v

def equilibrium(u, v, rho, f, ts):
    # also implements falcucci's collis method
    # maybe needs to return an object, but I think this should modify the original array
    for k in range(0,9):
        cu = (1./cs**2)*(ex[k]*u+ey[k]*v)
        feq = rho*w[k]*(1.+cu+cu**2-(u**2+v**2)/(2*cs**2))
        f[ts, k, :,:] = f[ts-1,k,:,:] +(1./omega)*(feq-f[ts-1,k,:,:])

def run():
# initialization
    f, rho, u, v = init(rho_in)
    
    # main loop
    for ts in range(1,Nsteps):
        # prog bar
        prog_bar(ts,Nsteps)
        
        # calc macro quanitites
        rho, u, v = calc_macro(f[ts-1],rho,u,v)

        # force + collision
        u, v = force(rho, u, v)
        equilibrium(u, v, rho, f, ts)


        # prepare the next time step
        fout = f[ts]

        # streaming
        for j in range(ly):
            for i in range(lx):
                for kk in range(0,9): 
                    f[ts,kk,i,j] = fout[kk,(i+ex[kk])%lx,(j+ey[kk])%ly]
    return f


# In[137]:





w = [4./9,1./9,1./9,1./9,1./9,1./36,1./36,1./36,1./36]
ex=[0,1,0,-1,0 ,1,-1,-1, 1]
ey=[0,0,1,0 ,-1,1, 1,-1,-1]
f_out = run()


# In[138]:


def calc_macro_f(f):
    #Assumes f at a given time step
    #  this fxn shouldn't be used in the loops as it takes times to create new numpy arrays
    # useful after running for getting macro quantities w/o having the rho,u, and v arrays handy
    u = np.zeros([lx,ly])
    v = np.zeros([lx,ly])
    
    rho = np.sum(f,axis=0)
    for k in range(9):
        u += f[k]*ex[k]
#         print(ey[k])
        v += f[k]*ey[k]
    u=u/rho
    v=v/rho
    return rho, u ,v

plt.imshow(calc_macro_f(f_out[1])[0])
plt.colorbar()
plt.grid('off')
plt.show()
plt.imshow(calc_macro_f(f_out[-1])[0])
plt.colorbar()
plt.grid('off')
plt.show()
finalfield=calc_macro_f(f_out[-1])[0]

average=np.mean(finalfield)
maxarg=np.argmax(finalfield)
maxpoint=[maxarg//lx,maxarg%lx]
count=0
while maxpoint[1]+count<lx and finalfield[maxpoint[0],maxpoint[1]+count]>average :
    count+=1
print("maximum radius",count)
minarg=np.argmin(finalfield)
minpoint=[minarg//lx,minarg%lx]
count=0
while minpoint[1]+count>0 and finalfield[minpoint[0],minpoint[1]+count]<average:
    count-=1
print("minimum radius",-count)






[==                  ] 17/300   

/home/ianhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:84: RuntimeWarning: overflow encountered in double_scalars
/home/ianhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:79: RuntimeWarning: overflow encountered in exp
/home/ianhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:85: RuntimeWarning: overflow encountered in double_scalars
/home/ianhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:86: RuntimeWarning: overflow encountered in double_scalars
/home/ianhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:87: RuntimeWarning: overflow encountered in double_scalars
/home/ianhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in double_scalars
/home/ianhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in double_scalars
/home/ianhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:84: RuntimeW

[==                  ] 19/300   

/home/ianhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:87: RuntimeWarning: invalid value encountered in double_scalars


[===                 ] 37/300   

KeyboardInterrupt: 